In [1]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd
import numpy as np

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def trade_option(symbol, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    start_time = start_time.replace(hour=9, minute=0, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)
     
    url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+symbol
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    
    return ohlc



In [2]:
symbol = 'BANKNIFTY'
days_before = 1
timeframe = 10
ohlc = trade_option(symbol, days_before, timeframe)

position = False
entry_price = None
position_type = None
total_profit_loss = 0
trades_taken = 0

# Assume 'ohlc' is your OHLC data DataFrame
ohlc['HA_Close'] = (ohlc['open'] + ohlc['high'] + ohlc['low'] + ohlc['close']) / 4
ha_open = ohlc['HA_Close'].shift(1)
ohlc['HA_Open'] = ha_open.values[0]
ohlc.loc[1:, 'HA_Open'] = ha_open.values[1:]
ohlc['HA_High'] = ohlc[['HA_Open', 'HA_Close', 'high']].max(axis=1)
ohlc['HA_Low'] = ohlc[['HA_Open', 'HA_Close', 'low']].min(axis=1)

for index, row in ohlc.iterrows():
    if index == 0:
        i = 1
    else:
        i = index
        
    if row['HA_Close'] >= row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']):
        if position and position_type == 'short':
            position = False
            profit_loss = entry_price - row['open']
            total_profit_loss += profit_loss
            print(f"Exit Sell: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

        if not position:
            position = True
            entry_price = row['open']
            position_type = "long"
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if row['HA_Close'] < row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']):
        if position and position_type == 'long':
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

        if not position:
            position = True
            entry_price = row['open']
            position_type = "short"
            trades_taken += 1
            print(f"Sell: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

# Print open trades
# if open_trade:
#     print("\nOpen Trades:")
#     print(f"Type: {open_trade['type']}, Entry Time: {unix_to_local_time(open_trade['entry_time'])}, Entry Price: {open_trade['entry_price']}")

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)} Total Options: {round(total_profit_loss*0.45,1)}")
if(symbol == "NIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*25)-(trades_taken*60)}")
if(symbol == "BANKNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*15)-(trades_taken*60)}")
if(symbol == "FINNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*40)-(trades_taken*60)}")
if(symbol == "NIFTYMIDSELECT"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*75)-(trades_taken*60)}")

Sell: Entry Time: 2024-05-28 09:35:00, Entry Price: 49358.75
Exit Sell: Time: 2024-05-28 10:15:00, Exit Price: 49308.45, P/L: 50.30000000000291
Long: Entry Time: 2024-05-28 10:15:00, Entry Price: 49308.45
Exit Long: Time: 2024-05-28 10:45:00, Exit Price: 49316.5, P/L: 8.05000000000291
Sell: Entry Time: 2024-05-28 10:45:00, Entry Price: 49316.5
Exit Sell: Time: 2024-05-28 12:55:00, Exit Price: 49276.7, P/L: 39.80000000000291
Long: Entry Time: 2024-05-28 12:55:00, Entry Price: 49276.7
Exit Long: Time: 2024-05-28 13:35:00, Exit Price: 49192.2, P/L: -84.5
Sell: Entry Time: 2024-05-28 13:35:00, Entry Price: 49192.2

Trades count: 5 cost: 300

Total Profit/Loss: 13.7 Total Options: 6.1

Take Away: -208.5


In [4]:
# Dynamic exit
symbol = 'NIFTY'
days_before = 7
timeframe = 10
ohlc = trade_option(symbol, days_before, timeframe)

position = False
entry_price = None
position_type = None
total_profit_loss = 0
trades_taken = 0

# Assume 'ohlc' is your OHLC data DataFrame
ohlc['HA_Close'] = (ohlc['open'] + ohlc['high'] + ohlc['low'] + ohlc['close']) / 4
ha_open = ohlc['HA_Close'].shift(1)
ohlc['HA_Open'] = ha_open.values[0]
ohlc.loc[1:, 'HA_Open'] = ha_open.values[1:]
ohlc['HA_High'] = ohlc[['HA_Open', 'HA_Close', 'high']].max(axis=1)
ohlc['HA_Low'] = ohlc[['HA_Open', 'HA_Close', 'low']].min(axis=1)

for index, row in ohlc.iterrows(): 
    if index == 0:
        i = 1
    else:
        i = index
        
    if row['HA_Close'] >= row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']) and (ohlc.iloc[i-1]['HA_Close']-ohlc.iloc[i-1]['HA_Open'] > 5):   
    #if row['HA_Close'] >= row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']):
        if not position:
            position = True
            entry_price = row['close']
            position_type = "long"
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")
    
    elif row['HA_Close'] < row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']) and (ohlc.iloc[i-1]['HA_Open']-ohlc.iloc[i-1]['HA_Close'] > 5):
    #elif row['HA_Close'] < row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']):
        if not position:
            position = True
            entry_price = row['close']
            position_type = "short"
            trades_taken += 1
            print(f"Sell: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")
    
    # Exit conditions
    if position:
        # Exit on opposite signal
        if position_type == 'long':
            if row['HA_Close'] < row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']):
                position = False
                profit_loss = row['close'] - entry_price
                total_profit_loss += profit_loss
                print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
                entry_price = None
    
        elif position_type == 'short':
            if row['HA_Close'] >= row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']):
                position = False
                profit_loss = entry_price - row['close']
                total_profit_loss += profit_loss
                print(f"Exit Sell: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
                entry_price = None
            

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)} Total Options: {round(total_profit_loss*0.45,1)}")
if(symbol == "NIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*25)-(trades_taken*60)}")
if(symbol == "BANKNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*15)-(trades_taken*60)}")
if(symbol == "FINNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*40)-(trades_taken*60)}")
if(symbol == "NIFTYMIDSELECT"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*75)-(trades_taken*60)}")

Sell: Entry Time: 2024-05-22 09:45:00, Entry Price: 22520.0
Exit Sell: Time: 2024-05-22 10:05:00, Exit Price: 22556.8, P/L: -38.400000000001455
Sell: Entry Time: 2024-05-22 11:05:00, Entry Price: 22546.3
Exit Sell: Time: 2024-05-22 11:25:00, Exit Price: 22537.6, P/L: -2.0499999999992724
Long: Entry Time: 2024-05-22 11:55:00, Entry Price: 22563.7
Exit Long: Time: 2024-05-22 12:35:00, Exit Price: 22572.45, P/L: -2.5499999999992724
Long: Entry Time: 2024-05-22 13:05:00, Entry Price: 22565.1
Exit Long: Time: 2024-05-22 14:15:00, Exit Price: 22575.25, P/L: 20.900000000001455
Sell: Entry Time: 2024-05-22 14:55:00, Entry Price: 22579.85
Exit Sell: Time: 2024-05-22 15:15:00, Exit Price: 22597.65, P/L: -42.25
Long: Entry Time: 2024-05-22 15:25:00, Entry Price: 22620.5
Exit Long: Time: 2024-05-23 10:05:00, Exit Price: 22664.95, P/L: 54.0
Long: Entry Time: 2024-05-23 10:25:00, Entry Price: 22693.4
Exit Long: Time: 2024-05-23 14:35:00, Exit Price: 22909.65, P/L: 234.84999999999854
Long: Entry Time